In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from collections import Counter
from sklearn.metrics import accuracy_score
import time

In [2]:
map_df = pd.read_csv('../Data/mapping_file.csv', index_col=0).sort_index()
map_df[:5]

,genotype,fertility,amf,rep,group,shoot_mass,root_mass,total_mass,ratio_root_to_shoot,root_colonization,...,Fe,Zn,Cu,Mn,Length,AvgDiam,Forks,SurfArea,Origin,Percent
#SampleID,,,,,,,,,,,,,,,,,,,,,
L1,1,P,Y,1,Common (C. dactylon),29.961,4.46,34.421,0.129572,43,...,205.131,40.296,24.655,134.694,162.8464,0.2905,904.0,14.8643,African,100%
L10,1,PPP,Y,2,Common (C. dactylon),26.283,10.16,36.443,0.278792,36,...,291.100,29.000,15.800,66.900,329.7942,0.2978,2620.0,30.8595,African,100%
L100,9,PP,Y,5,Common (C. dactylon),18.142,6.82,24.962,0.273215,21,...,48.811,38.705,9.024,55.564,229.4975,0.3495,1577.0,25.1969,Asian,100%
L101,9,PP,Y,1,Common (C. dactylon),25.649,10.88,36.529,0.297846,26,...,290.800,26.900,16.600,70.900,216.1839,0.3708,1395.0,25.1835,Asian,100%
L102,9,PP,Y,2,Common (C. dactylon),25.664,12.66,38.324,0.330341,39,...,265.700,29.200,20.700,83.900,302.0509,0.2743,1827.0,26.0297,Asian,100%


In [3]:
alpha_df = pd.read_csv('../Data/table.from_biom_w_taxonomy.txt',  skiprows =1, index_col=0, sep ='\t')
rf_df = alpha_df.T
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
map_df_numeric = map_df.select_dtypes(include=numerics)
rf_df.head()

#OTU ID,k__fungi;p__Glomeromycota;c__Glomeromycetes;o__Glomerales;f__Glomeraceae;g__Glomus;s__MO-G17,k__fungi;p__Glomeromycota;c__Glomeromycetes;o__Glomerales;f__Glomeraceae;g__Glomus;__,k__fungi;p__Glomeromycota;c__Glomeromycetes;o__Glomerales;f__Glomeraceae;g__Glomus;s__Yamato09 A2,k__fungi;p__Glomeromycota;c__Glomeromycetes;o__Glomerales;f__Glomeraceae;g__Glomus;s__Glo45,k__fungi;p__Glomeromycota;c__Paraglomeromycetes;o__Paraglomerales;f__Paraglomeraceae;g__Paraglomus;s__Alguacil12a Para 1,k__fungi;p__Glomeromycota;c__Glomeromycetes;o__Glomerales;f__Glomeraceae;g__Glomus;s__Whitfield type 17,k__fungi;p__Glomeromycota;c__Glomeromycetes;o__Glomerales;f__Glomeraceae;g__Glomus;s__GCL-6,k__fungi;p__Glomeromycota;c__Glomeromycetes;o__Glomerales;f__Glomeraceae;g__Glomus;s__MO-G8,k__fungi;p__Glomeromycota;c__Paraglomeromycetes;o__Paraglomerales;f__Paraglomeraceae;g__Paraglomus;__,k__fungi;p__Glomeromycota;c__Glomeromycetes;__;__;__;__,...,k__fungi;p__Glomeromycota;c__Glomeromycetes;o__Glomerales;f__Glomeraceae;g__Glomus;s__Glo3,k__fungi;p__Glomeromycota;c__Paraglomeromycetes;o__Paraglomerales;f__Paraglomeraceae;g__Paraglomus;s__IS-Pg1,k__fungi;p__Glomeromycota;c__Glomeromycetes;o__Glomerales;f__Glomeraceae;g__Glomus;s__Glo-D,k__fungi;p__Glomeromycota;c__Glomeromycetes;o__Glomerales;f__Glomeraceae;g__Glomus;s__MO-G27,k__fungi;p__Glomeromycota;c__Glomeromycetes;o__Glomerales;f__Glomeraceae;g__Glomus;s__acnaGlo2,k__fungi;p__Glomeromycota;c__Glomeromycetes;o__Glomerales;f__Glomeraceae;g__Glomus;s__cf. microaggregatum,k__fungi;p__Glomeromycota;c__Glomeromycetes;o__Glomerales;f__Claroideoglomeraceae;g__Claroideoglomus;s__Glo59,k__fungi;p__Glomeromycota;c__Glomeromycetes;o__Glomerales;f__Glomeraceae;g__Glomus;s__MO-G14,k__fungi;p__Glomeromycota;c__Glomeromycetes;o__Glomerales;f__Glomeraceae;g__Glomus;s__INTA-4,k__fungi;p__Glomeromycota;c__Glomeromycetes;o__Glomerales;f__Glomeraceae;g__Glomus;s__ORVIN GLO3B
L1,5556,515,457,519,0,33,0,153,57,0,...,0,0,0,0,0,0,0,0,0,0
L10,2649,524,0,0,61,170,0,0,270,0,...,0,0,0,0,0,0,0,0,0,0
L100,9868,161,51,186,7,0,0,50,249,0,...,0,0,0,0,0,0,0,0,0,0
L101,482,8626,364,189,3554,0,268,620,574,108,...,0,0,0,0,0,0,0,0,0,0
L102,9102,1029,5,114,0,28,0,523,57,9,...,0,0,0,0,0,0,0,0,0,0


In [4]:
print(rf_df.shape)
print(map_df.shape)

(156, 65)
(143, 28)


# Random forest for genotype

In [6]:
start = time.time()
accuracy_list = []
random_chance_list = []
for i in range(50):
    X_train, X_test, Y_train, Y_test = train_test_split(rf_df[rf_df.index.isin(map_df.index)].values,
                                                        map_df[map_df.index.isin(rf_df.index)]['genotype'].values, test_size=0.3, random_state=i)
    param_grid = { 
        'n_estimators': [200, 500],
        'max_features': ['auto', 'sqrt', 'log2'],
        'max_depth' : [2,3,4,5,6,7,8,9,10],
        'criterion' :['gini', 'entropy']
    }
    rfc=RandomForestClassifier(random_state=42)
    CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
    CV_rfc.fit(X_train, Y_train)
    predicted = CV_rfc.predict(X_test)
end = time.time()
print(end - start)

/home/bostevens/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/home/bostevens/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/home/bostevens/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/home/bostevens/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/home/bostevens/anac

8853.3425116539


In [7]:
print(CV_rfc.best_params_)

{'criterion': 'gini', 'max_depth': 9, 'max_features': 'log2', 'n_estimators': 200}


In [8]:
start = time.time()
accuracy_list = []
random_chance_list = []
predicted_category = 'genotype'
for i in range(50):
    X_train, X_test, Y_train, Y_test = train_test_split(rf_df[rf_df.index.isin(map_df.index)].values,
                                                        map_df[map_df.index.isin(rf_df.index)][predicted_category].values, test_size=0.3, random_state=i)
    clf = RandomForestClassifier(n_estimators=200, max_features='log2', max_depth=9, random_state=42, criterion= 'gini')
    clf.fit(X_train, Y_train)
    predicted = clf.predict(X_test)
    accuracy = accuracy_score(Y_test, predicted)
    accuracy_list.append(accuracy)
end = time.time()
print(end - start)

12.544850826263428


In [9]:
print(sum(accuracy_list)/len(accuracy_list)*100)

33.023255813953504


In [10]:
print(metrics.classification_report(Y_test, predicted))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00         5
           2       0.00      0.00      0.00         1
           3       0.17      0.50      0.25         2
           4       0.00      0.00      0.00         1
           5       0.00      0.00      0.00         3
           6       1.00      0.14      0.25         7
           7       0.00      0.00      0.00         4
           8       0.20      0.17      0.18         6
           9       0.00      0.00      0.00         3
          10       0.60      0.75      0.67         4
          11       0.33      0.20      0.25         5
          12       0.17      0.50      0.25         2

    accuracy                           0.19        43
   macro avg       0.21      0.19      0.15        43
weighted avg       0.30      0.19      0.18        43



/home/bostevens/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [11]:
# Labels are the values we want to predict
labels = map_df[map_df.index.isin(rf_df.index)][predicted_category].values
# Remove the labels from the features
# axis 1 refers to the columns
features= rf_df
# Saving feature names for later use
feature_list = list(features.columns)
# Convert to numpy array
features = np.array(features)
# Get numerical feature importances
importances = list(clf.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: k__fungi;p__Glomeromycota;c__Glomeromycetes;o__Glomerales;f__Glomeraceae;g__Glomus;__ Importance: 0.05
Variable: k__fungi;p__Glomeromycota;c__Glomeromycetes;o__Glomerales;f__Glomeraceae;g__Glomus;s__ORVIN GLO3D Importance: 0.05
Variable: k__fungi;p__Glomeromycota;__;__;__;__;__ Importance: 0.05
Variable: k__fungi;p__Glomeromycota;c__Glomeromycetes;o__Glomerales;f__Glomeraceae;g__Glomus;s__MO-G17 Importance: 0.04
Variable: k__fungi;p__Glomeromycota;c__Glomeromycetes;o__Glomerales;f__Glomeraceae;g__Glomus;s__Yamato09 A2 Importance: 0.04
Variable: k__fungi;p__Glomeromycota;c__Glomeromycetes;o__Glomerales;f__Glomeraceae;g__Glomus;s__Glo45 Importance: 0.04
Variable: k__fungi;p__Glomeromycota;c__Glomeromycetes;o__Glomerales;f__Glomeraceae;g__Glomus;s__MO-G8 Importance: 0.04
Variable: k__fungi;p__Glomeromycota;c__Paraglomeromycetes;o__Paraglomerales;f__Paraglomeraceae;g__Paraglomus;__ Importance: 0.04
Variable: k__fungi;p__Glomeromycota;c__Glomeromycetes;o__Glomerales;f__Glomeraceae

# Random forest for origin

In [ ]:
map_df = map_df[map_df['Origin'] != 'Unknown']

In [12]:
start = time.time()
accuracy_list = []
random_chance_list = []
for i in range(50):
    X_train, X_test, Y_train, Y_test = train_test_split(rf_df[rf_df.index.isin(map_df.index)].values,
                                                        map_df[map_df.index.isin(rf_df.index)]['Origin'].values, test_size=0.3, random_state=i)
    param_grid = { 
        'n_estimators': [200, 500],
        'max_features': ['auto', 'sqrt', 'log2'],
        'max_depth' : [2,3,4,5,6,7,8,9,10],
        'criterion' :['gini', 'entropy']
    }
    rfc=RandomForestClassifier(random_state=42)
    CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
    CV_rfc.fit(X_train, Y_train)
    predicted = CV_rfc.predict(X_test)
end = time.time()
print(end - start)

8195.688406705856


In [13]:
print(CV_rfc.best_params_)

{'criterion': 'entropy', 'max_depth': 5, 'max_features': 'log2', 'n_estimators': 500}


In [13]:
start = time.time()
accuracy_list = []
random_chance_list = []
predicted_category = 'Origin'
for i in range(50):
    X_train, X_test, Y_train, Y_test = train_test_split(rf_df[rf_df.index.isin(map_df.index)].values,
                                                        map_df[map_df.index.isin(rf_df.index)][predicted_category].values, test_size=0.3, random_state=i)
    clf = RandomForestClassifier(n_estimators=500, max_features='log2', max_depth=5, random_state=42, criterion= 'entropy')
    clf.fit(X_train, Y_train)
    predicted = clf.predict(X_test)
    accuracy = accuracy_score(Y_test, predicted)
    accuracy_list.append(accuracy)
end = time.time()
print(end - start)

23.2651526927948


In [14]:
print(sum(accuracy_list)/len(accuracy_list)*100)

75.3125


In [16]:
print(metrics.classification_report(Y_test, predicted))

              precision    recall  f1-score   support

     African       0.78      0.44      0.56        16
       Asian       0.61      0.88      0.72        16

    accuracy                           0.66        32
   macro avg       0.69      0.66      0.64        32
weighted avg       0.69      0.66      0.64        32



In [18]:
# Labels are the values we want to predict
labels = map_df[map_df.index.isin(rf_df.index)][predicted_category].values
# Remove the labels from the features
# axis 1 refers to the columns
features= rf_df
# Saving feature names for later use
feature_list = list(features.columns)
# Convert to numpy array
features = np.array(features)
# Get numerical feature importances
importances = list(clf.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: k__fungi;p__Glomeromycota;c__Glomeromycetes;o__Glomerales;f__Glomeraceae;g__Glomus;s__MO-G23 Importance: 0.12
Variable: k__fungi;p__Glomeromycota;c__Glomeromycetes;o__Glomerales;f__Glomeraceae;g__Glomus;s__MO-G8 Importance: 0.06
Variable: k__fungi;p__Glomeromycota;__;__;__;__;__ Importance: 0.06
Variable: k__fungi;p__Glomeromycota;c__Glomeromycetes;o__Glomerales;f__Glomeraceae;g__Glomus;s__MO-G22 Importance: 0.06
Variable: k__fungi;p__Glomeromycota;c__Paraglomeromycetes;o__Paraglomerales;f__Paraglomeraceae;g__Paraglomus;s__Alguacil12a Para 1 Importance: 0.05
Variable: k__fungi;p__Glomeromycota;c__Glomeromycetes;o__Glomerales;f__Glomeraceae;g__Glomus;s__Torrecillas12b Glo G13 Importance: 0.05
Variable: k__fungi;p__Glomeromycota;c__Glomeromycetes;o__Glomerales;f__Glomeraceae;g__Glomus;s__Yamato09 A2 Importance: 0.04
Variable: k__fungi;p__Glomeromycota;c__Glomeromycetes;o__Glomerales;f__Glomeraceae;g__Glomus;s__ORVIN GLO3D Importance: 0.04
Variable: k__fungi;p__Glomeromycota;c__